<a href="https://colab.research.google.com/github/AmbiTyga/Task-Stylumia/blob/main/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Kaggle Download
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c stumbleupon

Saving kaggle.json to kaggle.json
 99% 156M/157M [00:01<00:00, 96.1MB/s]
100% 157M/157M [00:01<00:00, 95.7MB/s]
  0% 0.00/21.5k [00:00<?, ?B/s]
100% 21.5k/21.5k [00:00<00:00, 17.7MB/s]
  0% 0.00/3.45M [00:00<?, ?B/s]
100% 3.45M/3.45M [00:00<00:00, 219MB/s]
 62% 5.00M/8.02M [00:00<00:00, 20.6MB/s]
100% 8.02M/8.02M [00:00<00:00, 26.7MB/s]


In [2]:
!unzip -q /content/raw_content.zip

In [3]:
!pip install boilerpy3 -q
!pip install scrapy -q

     |████████████████████████████████| 245kB 11.7MB/s 
     |████████████████████████████████| 245kB 15.6MB/s 
     |████████████████████████████████| 3.2MB 29.7MB/s 
     |████████████████████████████████| 3.2MB 54.6MB/s 
     |████████████████████████████████| 3.1MB 39.3MB/s 
     |████████████████████████████████| 61kB 5.4MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 81kB 7.2MB/s 


In [53]:
import pandas as pd 
import numpy as np
import json
from bs4 import BeautifulSoup
from boilerpy3 import extractors
import re

In [6]:
def extract_domain(df_url):
    # extract domains
    domain = df_url.str.lower().apply(lambda x:x.split('/')[2])
    domain_parts = domain.str.split('.')

    return domain_parts.apply(lambda x:'.'.join(x[-2:]) if x[-2] not in ['com', 'co'] else '.'.join(x[-3:]))

In [51]:
def clean_string(s):
    s = re.sub(r'[\n\r\t]', ' ',s)
    s = re.sub(r"\s+", ' ', s)
    return s.strip()

In [72]:
def get_boilerPipe(urlid):
  ext = extractors.ArticleExtractor()
  f = open("/content/raw_content/"+str(urlid), 'rb')
  s = ext.get_content(f.read())
  return clean_string(s)

In [5]:
data = pd.concat([pd.read_csv('/content/train.tsv.zip',sep = '\t'),
                  pd.read_csv('/content/test.tsv.zip',sep = '\t')],ignore_index = True)

In [7]:
item = data.copy()

In [8]:
item['real_url'] = item['url'].str.lower()

In [9]:
item['domain'] = extract_domain(item['real_url'])

In [10]:
item['tld'] = item['domain'].str.split('.').apply(lambda x:x[-1])

In [11]:
item['boilerplate'] = item['boilerplate'].apply(lambda x:json.loads(x))
item.rename(columns = {'url':'org_url'},inplace = True)

item = pd.concat([item,pd.DataFrame(item['boilerplate'].tolist(),columns=['title','body','url'])],axis = 1)
item.drop(columns = ['boilerplate'],inplace = True)

In [19]:
item['boilerplate'] = item[['title','body']].values.tolist()

In [23]:
def boil_soup(urlid, parser="lxml"):
    filename = '/content/raw_content/' + urlid

    with open(filename, 'rb') as f:
        html = f.read()

        for parser in ["lxml", "xml", "html5lib"]:
            soup = BeautifulSoup(html, parser)
            if soup.body:
                return soup

        return BeautifulSoup(html)

In [59]:
item['boilerpipe'] = item['urlid'].apply(get_boilerPipe)

UnicodeDecodeError: ignored

In [73]:
a = []
for i, id in enumerate(item['urlid'].values):
  print(i)
  a.append(get_boilerPipe(id))


0


TypeError: ignored

In [74]:
item['urlid'][0]

4042

In [80]:
f = open('/content/raw_content/3651','rb')

In [81]:
a = f.read()

In [82]:
type(a)

bytes

In [89]:
a.decode('utf-8')

UnicodeDecodeError: ignored

In [99]:

ext = extractors.ArticleExtractor()
clean_string(ext.get_content(soup.text))

''

In [98]:
soup.text

'\n\n\r\n\r\n/* Layout CSS MUST load locally on all pages*/\r\nbody {text-align: center;margin: 0;padding: 0; background-image:url(http://images.media-allrecipes.com/global/nav/topnav2/backgrounds/background.gif);}\r\n\t\r\n#topads {width: 998px;margin: 5px auto;text-align: center;}\r\n#wrap {width: 998px;margin: 0 auto;text-align: left;border: 2px solid #e6e4d7;border-bottom:none;}\r\n\r\n#main {float: left;width: 675px;}\r\n\r\n#farright {float: right;width: 310px;}\r\n#left {float: left;width: 170px;}\r\n#center {float: right;width: 505px;}\r\n#copyright \t{width: 998px; margin: 0 auto; padding: 6px 0px; color: #666666; font-family: Verdana, San-Serif; font-size: 9px;}\r\n\r\n.right {float: right;}\r\n.left {float: left;}\r\n\r\n.clearfix:after {content: "."; display: block; height: 0; clear: both; visibility: hidden;}\r\n\r\n.clearfix {display:inline-block;}\r\n/* Hide from IE Mac */\r\n.clearfix {display:block;}\r\n/* End hide from IE Mac */\r\n\r\n.navlist {list-style-type: none;